In [2]:
import random
from collections import Counter

# Our dataset
X = [
    [9.2, 85, 8],
    [8, 80, 7],
    [8.5, 81, 8],
    [6, 45, 5],
    [6.5, 50, 4],
    [8.2, 72, 7],
    [5.8, 38, 5],
    [8.9, 91, 9]
]
y = ["p", "p", "p", "f", "f", "p", "f", "p"]

def gini_index(groups, classes):
    """Calculate Gini impurity for split"""
    n_instances = sum([len(group) for group in groups])
    gini = 0.0
    for group in groups:
        size = len(group)
        if size == 0:
            continue
        score = 0.0
        labels = [row[-1] for row in group]
        for class_val in classes:
            p = labels.count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini

def test_split(index, value, dataset):
    """Split dataset based on feature index and threshold"""
    left, right = [], []
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

def get_split(dataset):
    """Choose the best split"""
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    n_features = len(dataset[0]) - 1
    for index in range(n_features):
        for row in dataset:
            groups = test_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index': b_index, 'value': b_value, 'groups': b_groups}

def to_terminal(group):
    """Create a terminal node value"""
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)

def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = get_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = get_split(right)
        split(node['right'], max_depth, min_size, depth+1)

def build_tree(train, max_depth, min_size):
    root = get_split(train)
    split(root, max_depth, min_size, 1)
    return root

def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']

def subsample(dataset, ratio=1.0):
    """Bootstrap sample"""
    sample = []
    n_sample = round(len(dataset) * ratio)
    while len(sample) < n_sample:
        index = random.randrange(len(dataset))
        sample.append(dataset[index])
    return sample

def random_forest(train, max_depth, min_size, n_trees):
    forest = []
    for _ in range(n_trees):
        sample = subsample(train)
        tree = build_tree(sample, max_depth, min_size)
        forest.append(tree)
    return forest

def bagging_predict(trees, row):
    preds = [predict(tree, row) for tree in trees]
    return Counter(preds).most_common(1)[0][0]

dataset = [X[i] + [y[i]] for i in range(len(X))]

forest = random_forest(dataset, max_depth=3, min_size=1, n_trees=5)

predictions = [bagging_predict(forest, row) for row in X]

print("Predictions:", predictions)
print("Actual:", y)

Predictions: ['p', 'f', 'p', 'f', 'f', 'p', 'f', 'p']
Actual: ['p', 'p', 'p', 'f', 'f', 'p', 'f', 'p']


In [6]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

# Dataset
X = np.array([
    [9.2, 85, 8],
    [8.0, 80, 7],
    [8.5, 81, 8],
    [6.0, 45, 5],
    [6.5, 50, 4],
    [8.2, 72, 7],
    [5.8, 38, 5],
    [8.9, 91, 9]
])
y = np.array(["p", "p", "p", "f", "f", "p", "f", "p"])

# Base learner (weak learner) = decision stump (max_depth=1)
base_tree = DecisionTreeClassifier(max_depth=1)

# AdaBoost classifier
model = AdaBoostClassifier(estimator=base_tree, n_estimators=10, random_state=42)
model.fit(X, y)

# Predictions on training data
predictions = model.predict(X)

print("Predictions:", predictions)
print("Actual:", list(y))
print("Accuracy:", model.score(X, y))

Predictions: ['p' 'p' 'p' 'f' 'f' 'p' 'f' 'p']
Actual: ['p', 'p', 'p', 'f', 'f', 'p', 'f', 'p']
Accuracy: 1.0


/home/student/anaconda3/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
